# Critical Incidents in Youth Mental Health Facilities in Utah

[Story](https://www.apmreports.org/story/2021/06/23/provo-canyon-school-utah-teen-concussion): Utah Found Few Rule Violations for Years<br /> 
Data Reporter: Will Craft, wcraft@apmreports.org<br /> 
Data Analysis Fact Checker: José Martínez, martinez307jose@gmail.com


"Up until 2019, the agency regulating Utah’s massive youth treatment industry rarely cited facilities for violating rules — even after cases of abuse. After a 2016 incident left a teenager with a concussion, state regulators listened to his mom’s complaint — and then did nothing about it."

"The lack of enforcement came to light thanks to a trove of never-before-seen records published this year by The Salt Lake Tribune, APM Reports and KUER. The three news organizations joined forces to publish five years of inspection reports and citations for rule violations from the 159 residential and wilderness programs that operated in Utah during that time period."

### Data Overview

The raw data was obtained through a records request made by APM Reports. Dozens of pdf files were scraped into the following spreadsheet: "cleaned_critical_incidents.csv"

The spreadsheet contains all the critical incidents reports filed by youth mental health facilities around Utah. The spreadsheet contains the following:
• Facility
• Investigation name
• Summary of the incident
• Conclusion
• Timeline concerning the start and end date of the investigation
• Date that the incident occurred.

With that information, we used pandas to filter investigations by facility, filter investigations by year, and filter investigations concerning sexual abuse or conduct by year.

Will Craft led the data analysis, and as a way to begin my internship with APM Reports, he had me fact check his work for this story so that I can warm up my data analysis skills.

It's important to note that the final chart in the story has different numbers than the original analysis. Reason being, the leading reporters added notice of agency actions from https://hslic.utah.gov/notices-of-agency-action. That is not reflected in this analysis.


In [1]:
import pandas as pd

In [2]:
import altair as alt

In [3]:
from collections import OrderedDict

In [4]:
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [6]:
df = pd.read_csv("/Users/josemartinez/Desktop/Code/utah/cleaned_critical_incidents.csv")

In [7]:
df.columns

Index(['facility_name', 'investigation_name', 'start_date', 'incident_date',
       'reported_date', 'finalized_date', 'summary', 'conclusion', 'file_path',
       'start_date_year', 'incident_date_year', 'reported_date_year',
       'finalized_date_year'],
      dtype='object')

# Below, I start figuring out informatino about sexual misconduct and abuse cases.

In [10]:
# Each row is an incident, so I'm figuring out how many incidents there are in total.
df.summary.count()

273

In [13]:
# I was interested in seeing how many incidents involved some sort of sexual abuse or misconduct,
# so I searched for the string in the investigation name.
df.investigation_name.str.contains('sexual',case=False,regex=False).sum()

36

In [265]:
# Percentage of investigations that involve sexual abuse or misconduct.
(36/273)*100

13.186813186813188

In [14]:
# However, I had forgotten to check if any values for empty, so I had to account for that
df.investigation_name.isna().sum()

11

In [15]:
# new dataframe with only investigation name and year that it started
sa=df[['investigation_name','start_date_year']]

In [269]:
#Wanted to see how many sexual abuse investigations there were in 2020
sa1 = sa[(sa['start_date_year']== 2020)].investigation_name.str.contains('sexual',case=False,regex=False).sum()

In [16]:
# Here, I learned a cool way to print both the analysis of each year and text to categorize it
print(f'''
     Investigations Related to Sexual Misconduct or Abuse in 2020 = {sa[(sa['start_date_year']== 2020)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}
     Investigations Related to Sexual Misconduct or Abuse in 2019 = {sa[(sa['start_date_year']== 2019)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}
     Investigations Related to Sexual Misconduct or Abuse in 2018 = {sa[(sa['start_date_year']== 2018)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}
     Investigations Related to Sexual Misconduct or Abuse in 2017= {sa[(sa['start_date_year']== 2017)].investigation_name.str.contains('sexual',case=False,regex=False).sum()}''')


     Investigations Related to Sexual Misconduct or Abuse in 2020 = 14
     Investigations Related to Sexual Misconduct or Abuse in 2019 = 19
     Investigations Related to Sexual Misconduct or Abuse in 2018 = 1
     Investigations Related to Sexual Misconduct or Abuse in 2017= 0


## Below, I start figuring out the number of incidents by facility

In [17]:
# here i see the amount of times each facility was mentioned in the doc.
# It seems provo canyon springville had the most incidents
df1 = df.facility_name.value_counts().rename_axis('Facilities').reset_index(name='Incidents')
df1

,Facilities,Incidents
0,provo_canyon_school_provo_canyon_springville__...,15
1,provo_canyon_school_provo_canyon_school___crit...,14
2,three_points_center_three_points_center___crit...,10
3,youth_health_associates_draper_ranch_yha_drape...,10
4,synergy_youth_treatment_cornish_synergy_cornis...,9
...,...,...
88,turn_about_ranch_barn_turn_about_ranch_barn___...,1
89,lifeline_behavioral_health_lifeline___critical...,1
90,crossroads_academy_south_crossroads_south___cr...,1
91,liahona_academy_for_youth_virgin_liahona_virgi...,1


In [62]:
df1.shape

(93, 2)

In [19]:
# Was looking to just see the top five facilities with the most incidents
data = df1[:5]
data

,Facilities,Incidents
0,provo_canyon_school_provo_canyon_springville__...,15
1,provo_canyon_school_provo_canyon_school___crit...,14
2,three_points_center_three_points_center___crit...,10
3,youth_health_associates_draper_ranch_yha_drape...,10
4,synergy_youth_treatment_cornish_synergy_cornis...,9


In [21]:
data1=data.copy()

In [22]:
# Cleaning time!!! Each value has underscores, isnt' capitalized, repeats certain phrases, so it was time to clean.
data1['Facilities'] = data1['Facilities'].str.replace('__critical_incidents', '')

In [118]:
data1['Facilities']=data1['Facilities'].str.replace('_',' ')

In [119]:
data1['Facilities']=data1['Facilities'].str.title()

In [121]:
data1['Facilities']=data1['Facilities'].str.replace("Provo Canyon School",'',1)

In [123]:
data1['Facilities']=data1['Facilities'].str.replace('Three Points Center','',1)

In [125]:
data1['Facilities']=data1['Facilities'].str.replace('Synergy Cornish','',1)

In [127]:
data1['Facilities']=data1['Facilities'].str.replace('Yha Draper Ranch','',1)

In [128]:
# Clean data looks great. Nice
data1

,Facilities,Incidents
0,Provo Canyon Springville,15
1,Provo Canyon School,14
2,Youth Health Associates Draper Ranch,10
3,Three Points Center,10
4,Synergy Youth Treatment Cornish,9


In [23]:
# My first ever attempt at visualization!! Will uses altair, so that's where I started.
alt.Chart(data1, title="Critical Incidents by Facility").mark_bar().encode(
x=alt.X('Facilities',axis=alt.Axis(labelAngle=0),sort='y'),
y='Incidents',
).properties(
    width=1000,
    height=500)

alt.Chart(...)

## Here I start figuring out the number of incidents per year at Provo Canyon Only.

In [25]:
# new dataframe
count = df[['facility_name','start_date_year']]

In [26]:
# only wanted values for provo canyon
prov = count[count['facility_name']=='provo_canyon_school_provo_canyon_school___critical_incidents']

In [27]:
prov1=prov.copy()

In [28]:
# Cleaning time again!!
prov1['facility_name']=prov1['facility_name'].str.replace('__critical_incidents', '')

In [29]:
prov1['facility_name']=prov1['facility_name'].str.replace('_',' ')

In [30]:
prov1['facility_name']=prov1['facility_name'].str.title()

In [31]:
prov1['facility_name']=prov1['facility_name'].str.replace("Provo Canyon School",'',1)

In [32]:
# Wanted to see how many incidents per year there were at Provo
prov2 = prov1.start_date_year.value_counts().rename_axis('Year').reset_index(name='Incidents')
prov2

,Year,Incidents
0,2019.0,7
1,2017.0,3
2,2020.0,2
3,2018.0,2


In [33]:
prov2.columns

Index(['Year', 'Incidents'], dtype='object')

In [34]:
prov2['Year'] = prov2['Year'].astype(str)

In [35]:
prov2['Year'] = prov2['Year'].str.replace('.','',1)

<ipython-input-35-733ac7278971>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  prov2['Year'] = prov2['Year'].str.replace('.','',1)


In [36]:
prov2['Year'] = prov2['Year'].str[:-1]

In [37]:
prov2

,Year,Incidents
0,2019,7
1,2017,3
2,2020,2
3,2018,2


In [525]:
alt.Chart(prov2, title='Provo Incidents by Year').mark_bar().encode(
x=alt.X('Year',axis=alt.Axis(labelAngle=0)),y='Incidents',).properties(
    width=500,
    height=500)

alt.Chart(...)

# Here I figure out the total # of incidents per year at all facilities.

In [59]:
dfstart=df['start_date_year'] # the reason i use start date is because some investigations might
#not be finished, so they wouldn't appear in the finalzied dat year.

In [60]:
# total number of investigations by year
dfstart.value_counts()

2019.0    126
2020.0    112
2018.0     21
2017.0      9
Name: start_date_year, dtype: int64

# Below, I'm curious as to how many empty values there are per category.

In [47]:
df.reported_date_year.isna().sum()

70

In [48]:
df.start_date_year.isna().sum()

5

In [49]:
df.finalized_date_year.isna().sum()

3

In [50]:
print(f'''
     Reported Date Year NA = {df.reported_date_year.isna().sum()}
     Start Date Year NA = {df.start_date_year.isna().sum()}
     Finalized Date Year NA = {df.finalized_date_year.isna().sum()}
     Incident Date Year NA = {df.incident_date_year.isna().sum()}''')


     Reported Date Year NA = 70
     Start Date Year NA = 5
     Finalized Date Year NA = 3
     Incident Date Year NA = 72
